### INIT

In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from utils.utils_chain import WrapperAddress


SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"

GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)
DOCKER_URL = PROJECT_ROOT / "docker"

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

wasm_path = config.DEFAULT_WORKSPACE / "wasm/fees-collector.wasm"
contract_code_hash = "08cd6ef3e0d5be0c7d4aedc2740d06f21b0aad28080ada2a551778af454176fe"

### CHAIN SIM CONFIG - FEES COLLECTOR

In [2]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from argparse import Namespace

docker_path = config.CHAIN_SIMULATOR_DOCKER_PATH
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))


In [3]:
USERS = [
        "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d",
        "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw", 
        "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm",
        "erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n",
        "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97" # DEX OWNER
]

BASE_TOKEN = "MEX-455c57"

In [4]:
import json
import subprocess
from time import sleep
from typing import Any

from utils.utils_chain import Account


def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    
def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", state)

# @pytest.fixture
def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def advance_blocks(number_of_blocks: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})

def users_init(found_users: list[str]) -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in found_users:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

In [ ]:
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from contracts.router_contract import RouterContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

config.DEFAULT_PROXY =  "https://gateway.multiversx.com"

energy_contract: SimpleLockEnergyContract
router_contract: RouterContract
pair_contract: PairContract

pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract = context.deploy_structure.get_deployed_contract_by_index(config.ROUTER_V2, 0)

print(fees_collector_contract.address)
print(router_contract.address)
print(pair_contract.address)
print(mex_contract.address)


In [6]:
from utils.utils_chain import base64_to_hex
from utils.utils_chain import WrapperAddress


def fees_collector_upgrade(bytecode_path: Path, expected_code_hash: str = None):
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, 
                                         args, 
                                         no_init=True)

    chain_sim.advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
    if expected_code_hash:
        assert code_hash == expected_code_hash
    else:
        print(f"Code hash: {code_hash}")
    
    return tx_hash

In [7]:
from multiversx_sdk.abi import Abi

abi = Abi.load(config.HOME / "Projects/dex/mx-exchange-sc/output-docker/fees-collector/fees-collector.abi.json")
    
def build_args_for_swap():

    routes: list[list[Any]] = []
    routes.append((Address(pair_contract.address), "swapTokensFixedInput" , pair_contract.firstToken, 1))
    routes.append((Address(mex_contract.address), "swapTokensFixedInput" , "MEX-455c57", 1))
    
    sc_args = [
           pair_contract.secondToken,
           routes
        ]

    return sc_args

In [8]:
def get_next_week_start_epoch() -> int:
    first_week = fees_collector_contract.get_first_week_start_epoch(context.network_provider.proxy)
    current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    next_week_at_epoch = first_week + current_week * 7

    print(f"Current epoch: {context.network_provider.proxy.get_network_status().current_epoch}")
    print(f"Current week: {current_week}")
    print(f"Next week at epoch: {next_week_at_epoch}")

    return next_week_at_epoch

# CHAIN CONTROL

In [ ]:
chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')

In [ ]:
chain_sim.stop()

### CONTRACTS

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address

users = users_init()
user_account = users[0]
user_account.sync_nonce(context.network_provider.proxy)
print(user_account.address.to_bech32())

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_fees_collectors_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
chain_sim.advance_epochs(7)

In [ ]:
chain_sim.advance_blocks(5)

CLAIM REWARDS

In [ ]:
user_with_energy = users[1]
print(user_with_energy.address.to_bech32())
energy_contract.get_energy_for_user(context.network_provider.proxy, user_with_energy.address.to_bech32())

In [ ]:
user_without_energy = users[0]
print(user_without_energy.address.to_bech32())
energy_contract.get_energy_for_user(context.network_provider.proxy, user_without_energy.address.to_bech32())

In [ ]:
user_with_energy.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
user_without_energy.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

CLAIM BOOSTED

In [ ]:
fees_collector_contract.claim_boosted_rewards(user_account, proxy)

ADD ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

REMOVE ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.remove_admin(context.deployer_account, proxy, [context.deployer_account.address])

REDISTRIBUTE REWARDS

In [ ]:

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.redistribute_rewards(context.deployer_account, proxy)
advance_blocks(1)

SET ADMINS

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

In [ ]:
week = fees_collector_contract.get_current_week(proxy)
user_energy = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
last_active_week = fees_collector_contract.get_last_active_week_for_user(user_account.address.to_bech32(), proxy)
print(week)
print(user_energy)
print(last_active_week)

GET TOTAL REWARDS FOR WEEK

In [ ]:
from utils.utils_chain import WrapperAddress, decode_merged_attributes
from utils import decoding_structures

data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
total_rewards = fees_collector_contract.get_total_rewards_for_week(proxy, week)
print(total_rewards)


GET ACCUMULATED FEES

In [ ]:
from multiversx_sdk.abi import TokenIdentifierValue, U32Value
token_identifier = "MEX-455c57"
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
accumulated_fees = data_fetcher.get_data("getAccumulatedFees", [U32Value(week), TokenIdentifierValue(token_identifier)])
print(accumulated_fees)


GET USER ENERGY FOR WEEK

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
user_energy_for_week = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
print(user_energy_for_week)


GET CURRENT CLAIM PROGRESS

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
current_claim = fees_collector_contract.get_current_claim_progress_for_user(user_account.address.to_bech32(), proxy)
print(current_claim)

In [ ]:
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, user_account, event)

In [ ]:
from utils.utils_chain import base64_to_hex


code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
print(code_hash)

PAIR SWAP

In [ ]:
from utils.utils_chain import get_token_details_for_address
from contracts.pair_contract import PairContract, SwapFixedInputEvent

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.secondToken, user_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {user_account.address.to_bech32()}")
print(f'Amount found on {user_account.address.to_bech32()}: {amount}')

user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.secondToken, amount//2, pair_contract.firstToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
chain_sim.advance_blocks(5)

In [ ]:
from utils.utils_chain import get_token_details_for_address
from contracts.pair_contract import PairContract, SwapFixedInputEvent

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.firstToken, user_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {user_account.address.to_bech32()}")
print(f'Amount found on {user_account.address.to_bech32()}: {amount}')

user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.firstToken, amount, pair_contract.secondToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
chain_sim.advance_blocks(5)

CONTRACT UPGRADE

In [10]:
from utils.utils_chain import base64_to_hex
from utils.utils_chain import WrapperAddress


def fees_collector_upgrade(bytecode_path: Path):
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, 
                                         args, 
                                         no_init=True)

    chain_sim.advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
    # assert code_hash == contract_code_hash
    
    return tx_hash

SET ROUTER ADDRESS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)

SET BURN PERCENTAGES

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

In [ ]:
pair_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
chain_sim.advance_blocks(1)

In [ ]:
print(pair_contract.get_config_dict())

SWAP TO BASE TOKEN

In [10]:
from multiversx_sdk.abi import Abi

abi = Abi.load(config.HOME / "Projects/dex/mx-exchange-sc/output-docker/fees-collector/fees-collector.abi.json")
    
def build_args_for_swap():

    routes: list[list[Any]] = []
    routes.append((Address(pair_contract.address), "swapTokensFixedInput" , pair_contract.firstToken, 1))
    routes.append((Address(mex_contract.address), "swapTokensFixedInput" , "MEX-455c57", 1))
    
    sc_args = [
           pair_contract.secondToken,
           routes
        ]

    return sc_args

In [ ]:
fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
chain_sim.advance_blocks(1)

In [ ]:
chain_sim.advance_blocks(1)

ADD REWARD TOKENS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_reward_tokens(context.deployer_account, proxy, ["MEX-455c57", "XMEX-fda355"])

REMOVE REWARD TOKENS

In [ ]:
fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, ["USDC-c76f1f",
                                                                               "RIDE-7d18e9", 
                                                                               "CRU-a5f4aa", 
                                                                               "ZPAY-247875", 
                                                                               "ITHEUM-df6f26", 
                                                                               "BHAT-c1fde3", 
                                                                               "CRT-52decf", 
                                                                               "UTK-2f80e9",
                                                                               "QWT-46ac01",
                                                                               "ASH-a642d1",
                                                                               "WETH-b4ca29",
                                                                               "USDT-f8c08c",
                                                                               "HTM-f51d55",
                                                                               "WDAI-9eeb54",
                                                                               "TADA-5c032c",
                                                                               "XOXNO-c1293a",
                                                                               "A1X-0d446d",
                                                                               "USH-111e09",
                                                                               "FOXSY-5d5f3e"
                                                                               ])
advance_blocks(1)

In [ ]:
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

context.deployer_account.sync_nonce(proxy)
tx_hash = energy_factory.set_burn_role_locked_token(context.deployer_account,
                                                                proxy,
                                                                [fees_collector_contract.address])

advance_blocks(1)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [mex_contract.address])
advance_blocks(1)

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalMint", "ESDTRoleLocalBurn"])


FUND USER

In [22]:
user = "erd1pye4dsy3fs956skp8wgf5pjtfynvw5y52gwdmsrrum9jcv3y2vusk7we6t"
chain_sim.apply_states([[{
        "address": user,
        "nonce": 0,
        "balance": "100000000000000000000",
        "username": "",
        "code": "",
        "developerReward": "0",
        "ownerAddress": "",
        "pairs": {}
    }]])

# SCENARIOS

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from argparse import Namespace
from time import sleep

docker_path = config.CHAIN_SIMULATOR_DOCKER_PATH
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from contracts.builtin_contracts import ESDTContract

# INITIALIZE USERS

USERS = found_accounts
users = users_init()

purse_account = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_without_energy = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_with_energy = next(user for user in users if user.address.to_bech32() == "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
user_with_energy_2 = next(user for user in users if user.address.to_bech32() == "erd1adljw932qra4sf5mpxjyzelmf4lykwt5ppxlre59utjcpc22uhms2qxcqx")

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.secondToken, purse_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {purse_account.address.to_bech32()}")
print(f'Amount found on {purse_account.address.to_bech32()}: {amount}')

# GET KNOWN CONTRACTS BEFORE UPGRADE -- THESE WILL BE REMOVED BY THE UPGRADE
known_contracts = fees_collector_contract.get_known_contracts(proxy)

# UPGRADE CONTRACT

fees_collector_upgrade()

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)

fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
chain_sim.advance_blocks(1)

# BEGIN SCENARIO

def week_activity():
    # CLAIM REWARDS 1

    user_with_energy.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    user_without_energy.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    # SWAP

    purse_account.sync_nonce(proxy)
    swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)

    pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
    chain_sim.advance_blocks(5)

    # CONVERT TO MEX
    args = build_args_for_swap()
    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
    chain_sim.advance_blocks(1)

    # CLAIM REWARDS 2

    user_with_energy_2.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
    chain_sim.advance_blocks(5)

# FIRST WEEKS ------------------------------------------------------------

weeks = 5
for week in range(weeks):

    print(f"Week {week}")

    # WEEK ACTIVITY
    week_activity()

    # PASS THE WEEK
    chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch())

# WEEK 5 SETUP ------------------------------------------------------------

user_with_energy.sync_nonce(context.network_provider.proxy)
fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

# CLEANUP KNOWN TOKENS

tokens = fees_collector_contract.get_reward_tokens(proxy)
tokens.remove(energy_contract.base_token)
tokens.remove(energy_contract.locked_token)

fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, tokens)
chain_sim.advance_blocks(1)

# SET NEW BURN PERCENTAGE

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

chain_sim.advance_blocks(5)

fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

# MODIFY FEES FOR KNOWN CONTRACTS

for address in known_contracts:
    mod_contract = PairContract.load_contract_by_address(address)
    if mod_contract is None:
        continue
    mod_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
    chain_sim.advance_blocks(1)

# WEEK ACTIVITY
week_activity()

chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch())

# WEEK ACTIVITY
week_activity()


BINARY COMPARE

In [11]:
from utils.utils_scenarios import PhaseDictsCollector
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from contracts.builtin_contracts import ESDTContract
from argparse import Namespace
from time import sleep

def test_scenario(bytecode_path: Path, collector: PhaseDictsCollector):
    users = users_init(found_accounts)

    purse_account = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
    user_without_energy = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
    user_with_energy = next(user for user in users if user.address.to_bech32() == "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
    user_with_energy_2 = next(user for user in users if user.address.to_bech32() == "erd1adljw932qra4sf5mpxjyzelmf4lykwt5ppxlre59utjcpc22uhms2qxcqx")

    pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
    print(pair_contract.get_config_dict())

    _, amount, _ = get_token_details_for_address(pair_contract.secondToken, purse_account.address.to_bech32(), context.network_provider.proxy)
    if amount == 0:
        raise Exception(f"No amount found on {purse_account.address.to_bech32()}")
    print(f'Amount found on {purse_account.address.to_bech32()}: {amount}')

    # GET KNOWN CONTRACTS BEFORE UPGRADE -- THESE WILL BE REMOVED BY THE UPGRADE
    known_contracts = fees_collector_contract.get_known_contracts(proxy)

    # UPGRADE CONTRACT

    fees_collector_upgrade(bytecode_path)

    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
    chain_sim.advance_blocks(1)

    fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
    chain_sim.advance_blocks(1)

    # BEGIN SCENARIO

    def week_activity(week: int):
        # CLAIM REWARDS 1

        user_with_energy.sync_nonce(context.network_provider.proxy)
        hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
        chain_sim.advance_blocks(5)
        sleep(2)
        claim_ops = context.network_provider.get_tx_operations(hash, True)
        collector.add(f"CLAIM_USER_1_W_ENERGY_WEEK_{week}", claim_ops, "Claim from user1 with energy")

        user_without_energy.sync_nonce(context.network_provider.proxy)
        fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
        chain_sim.advance_blocks(5)

        # SWAP

        purse_account.sync_nonce(proxy)
        swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)

        pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
        chain_sim.advance_blocks(5)

        # CONVERT TO MEX
        args = build_args_for_swap()
        context.deployer_account.sync_nonce(proxy)
        fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
        chain_sim.advance_blocks(1)

        # CLAIM REWARDS 2

        user_with_energy_2.sync_nonce(context.network_provider.proxy)
        hash = fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
        chain_sim.advance_blocks(5)
        sleep(2)
        claim_ops = context.network_provider.get_tx_operations(hash, True)
        collector.add(f"CLAIM_USER_2_W_ENERGY_WEEK_{week}", claim_ops, "Claim from user2 with energy")

    # FIRST WEEKS ------------------------------------------------------------

    weeks = 5
    for week in range(weeks):

        print(f"Week {week}")

        # WEEK ACTIVITY
        week_activity(week)

        # PASS THE WEEK
        chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch())

    # WEEK 5 SETUP ------------------------------------------------------------

    user_with_energy.sync_nonce(context.network_provider.proxy)
    hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)
    sleep(2)
    claim_ops = context.network_provider.get_tx_operations(hash, True)
    collector.add(f"RECLAIM_USER_1_W_ENERGY_WEEK_{week+1}", claim_ops, "Early claim from user1 with energy")

    # CLEANUP KNOWN TOKENS

    tokens = fees_collector_contract.get_reward_tokens(proxy)
    tokens.remove(energy_contract.base_token)
    tokens.remove(energy_contract.locked_token)

    fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, tokens)
    chain_sim.advance_blocks(1)

    # SET NEW BURN PERCENTAGE

    esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
    esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                                [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

    chain_sim.advance_blocks(5)

    fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
    chain_sim.advance_blocks(1)

    # MODIFY FEES FOR KNOWN CONTRACTS

    for address in known_contracts:
        mod_contract = PairContract.load_contract_by_address(address)
        if mod_contract is None:
            continue
        mod_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
        chain_sim.advance_blocks(1)

    # WEEK ACTIVITY
    week_activity(week+1)

    chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch())

    # WEEK ACTIVITY
    week_activity(week+2)

In [ ]:
upgrade1_path = config.HOME / "wasm/fees-collector.wasm"
upgrade2_path = config.HOME / "wasm/fees-collector-rewards-per-epoch.wasm"

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
    
collector = PhaseDictsCollector()

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

collector.set_phase("wasm-1")
test_scenario(upgrade1_path, collector)

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

collector.set_phase("wasm-2")
test_scenario(upgrade2_path, collector)

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")

In [ ]:
collector.print_collections()

In [ ]:
args = build_args_for_swap()
fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
chain_sim.advance_blocks(1)

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)

tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalBurn"])

pair_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
mex_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

purse_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)
pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
chain_sim.advance_blocks(5)

# CONVERT TO MEX

args = build_args_for_swap()
fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
chain_sim.advance_blocks(1)

# CLAIM REWARDS 2

user_with_energy_2.sync_nonce(context.network_provider.proxy)
fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
chain_sim.advance_blocks(1)       

In [ ]:
chain_sim.advance_epochs(7)

In [ ]:
chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch())